In [1]:
import pandas as pd
from ast import literal_eval as make_list
import networkx as nx

In [2]:
entire_path = '/home/suraj/Work/llnl/CallFlow/data/processed/lulesh-3-runs/ensemble_entire_df.csv'
group_path = '/home/suraj/Work/llnl/CallFlow/data/processed/lulesh-3-runs/ensemble_group_df.csv'
datasets = ['1-core', '27-cores', '512-cores']

In [3]:
entire_df = pd.read_csv(entire_path)
group_df = pd.read_csv(group_path)

In [4]:
group_path_df = group_df.groupby(['name', 'dataset'])['group_path', 'path']

In [5]:
for dataset in datasets:
    a = group_path_df.get_group(('ApplyMaterialPropertiesForElems', dataset))['path'].unique()
    b = group_path_df.get_group(('ApplyMaterialPropertiesForElems', dataset))['group_path'].unique()

    print(a, b)

["['main', 'lulesh.cycle', 'LagrangeLeapFrog', 'LagrangeElements', 'ApplyMaterialPropertiesForElems']"] ["['Lulesh=main', 'LeapFrog=LagrangeLeapFrog', 'CalcLagrange=LagrangeElements']"]
["['main', 'lulesh.cycle', 'LagrangeLeapFrog', 'LagrangeElements', 'ApplyMaterialPropertiesForElems']"] ["['Lulesh=main', 'LeapFrog=LagrangeLeapFrog', 'CalcLagrange=LagrangeElements']"]
["['main', 'lulesh.cycle', 'LagrangeLeapFrog', 'LagrangeElements', 'ApplyMaterialPropertiesForElems']"] ["['Lulesh=main', 'LeapFrog=LagrangeLeapFrog', 'CalcLagrange=LagrangeElements']"]


In [20]:
def get_group(g, key):
    if key in g.groups: return g.get_group(key)
    return pd.DataFrame()

In [21]:
def check_path_equivalence_across_runs(group_path_df, callsite, path_attr):
    paths = {}
    for dataset in datasets:
        # Assuming all ranks have the same path. 
        print(callsite, dataset)
        group = get_group(group_path_df, (callsite, dataset))
        print(group)
        key = group[path_attr].unique()[0]
        print(key)
        if key not in paths:
            paths[key] = []
        paths[key].append(dataset)

In [22]:
g = nx.DiGraph()
agg_g = nx.DiGraph()

In [25]:
def add_paths(path_attr='group_path'):
    ensemble_callsites = group_df['name'].unique()
    group_path_df = group_df.groupby(['name', 'dataset'])[path_attr].unique()
    for callsite in ensemble_callsites:
        check_path_equivalence_across_runs(group_path_df, callsite, path_attr)

In [26]:
add_paths()

main 1-core


AttributeError: 'Series' object has no attribute 'groups'

In [ ]:
def add_paths(path):
    print(self.group_df[['dataset', 'name', 'module', 'group_path']])
    paths = self.group_df[path].unique()

        for idx, path_str in enumerate(paths):
            path_list = self.rename_cycle_path(path_str)

            for callsite_idx, callsite in enumerate(path_list):
                if callsite_idx != len(path_list) - 1:
                    source = path_list[callsite_idx]
                    target = path_list[callsite_idx + 1]

                    print(source['module'], target['module'], source['callsite'], target['callsite'])

                    source_module = source['module']
                    target_module = target['module']

                    source_name = source['callsite']
                    target_name = target['callsite']

                    if(self.g.has_edge(target['module'], source['module'])):
                        edge_type = 'callback'
                    else:
                        edge_type = 'normal'

                    if(edge_type == 'normal'):
                        self.g.add_edge(source_module, target_module,   attr_dict={
                            "source_callsite": source_name,
                            "target_callsite": target_name,
                            "edge_type": edge_type
                        })